In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
import joblib

In [2]:
# Load the data and model
customer_product_matrix = pd.read_csv('../data/processed/customer_product_matrix.csv', index_col=0)
model = joblib.load('../models/buy_again_model.joblib')
scaler = joblib.load('../models/buy_again_scaler.joblib')

In [3]:
# Prepare the data
X = customer_product_matrix.values
X_scaled = scaler.transform(X)

In [4]:
# Function to get recommendations
def get_recommendations(customer_id, n_recommendations=5):
    customer_vector = X_scaled[customer_product_matrix.index == customer_id]
    distances, indices = model.kneighbors(customer_vector)
    
    similar_customers = customer_product_matrix.index[indices[0][1:]]
    recommended_products = customer_product_matrix.loc[similar_customers].sum().sort_values(ascending=False)
    
    return recommended_products.head(n_recommendations)

In [5]:
# Evaluate the model
def evaluate_recommendations(n_recommendations=5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    for customer_id in customer_product_matrix.index:
        actual_purchases = set(customer_product_matrix.columns[customer_product_matrix.loc[customer_id] > 0])
        recommendations = set(get_recommendations(customer_id, n_recommendations).index)
        
        true_positives += len(actual_purchases.intersection(recommendations))
        false_positives += len(recommendations - actual_purchases)
        false_negatives += len(actual_purchases - recommendations)
    
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1 = 2 * (precision * recall) / (precision + recall)
    
    return precision, recall, f1

In [6]:
# Evaluate the model
precision, recall, f1 = evaluate_recommendations()
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
# Example recommendations for a few customers
for customer_id in customer_product_matrix.index[:5]:
    print(f"\nRecommendations for customer {customer_id}:")
    print(get_recommendations(customer_id))


Recommendations for customer 12346.0:
10002    0.0
72134    0.0
72586    0.0
72598    0.0
72709    0.0
dtype: float64

Recommendations for customer 12347.0:
21064    50.0
23422    12.0
23420    12.0
22196    12.0
20936    12.0
dtype: float64

Recommendations for customer 12348.0:
21983    144.0
23076    120.0
23077    120.0
23230     50.0
84077     48.0
dtype: float64

Recommendations for customer 12349.0:
23273    44.0
22086    40.0
23202    40.0
23203    40.0
22077    36.0
dtype: float64

Recommendations for customer 12350.0:
23293     56.0
23232     50.0
22704     50.0
16156S    50.0
23120     48.0
dtype: float64
